In [ ]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [ ]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'
pattern_numbers = r'[^a-zA-Z0-9]'

In [ ]:
import string
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import numpy as np
import pygwalker as pyg
import googletrans
from googletrans import Translator
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tag import pos_tag, pos_tag_sents
from spacy.pipeline import Sentencizer


In [ ]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_dropped.pkl').reset_index(drop=True)


In [ ]:
len(df_jobs)


In [ ]:
df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
    lambda job_description: ' '.join(job_description.split('/')) if '/' in job_description else job_description
)


In [ ]:
abb_dict = {
    r'incl\.': 'including', 
    r'e\.g\.': 'for example', 
    r'e\.g': 'for example', 
    r'etc\.': 'et cetera', 
}


In [ ]:
str_fix_incl = 'Apply appropriate and effective communication methods to senior management and important stakeholders'
str_fix_eg = 'Partner with Procurement in order to manage suppliers for the projects & programs in scope'



In [ ]:
df_jobs['Job Description'] = df_jobs['Job Description'].replace(abb_dict, regex=True)

In [ ]:
df_jobs['Job Description'][df_jobs['Job Description'].str.contains(str_fix_incl)]


In [ ]:
re.findall(str_fix_eg, df_jobs['Job Description'][35])

In [ ]:
list(re.finditer(str_fix_eg, df_jobs['Job Description'][35]))

In [ ]:
df_jobs['Job Description'][35][1461:1580]

In [ ]:
# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('incl.', 'including')
# )

# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('e.g.', 'e.g')
# )

# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('e.g', 'for example')
# )

# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('etc.', 'et cetera')
# )


In [ ]:
# df_jobs = df_jobs[:100]

In [ ]:
# df_jobs = df_jobs.sample(n=100, random_state=1)

In [ ]:
# df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_glob_paths_10.pkl').reset_index(drop=True)


In [ ]:
df_jobs.info()


In [ ]:
dutch_requirement_pattern = r'[Ll]anguage: [Dd]utch|[Dd]utch [Pp]referred|[Dd]utch [Re]quired|[Dd]utch [Ll]anguage|[Pp]roficient in [Dd]utch|[Ss]peak [Dd]utch|[Kk]now [Dd]utch'
english_requirement_pattern = r'[Ll]anguage: [Ee]nglish|[Ee]nglish [Pp]referred|[Ee]nglish [Re]quired|[Ee]nglish [Ll]anguage|[Pp]roficient in [Ee]nglish|[Ss]peak [Ee]nglish|[Kk]now [Ee]nglish'


In [ ]:
# Language requirements
# Dutch
if 'Dutch Requirement' in df_jobs.columns:
    df_jobs.drop(columns=['Dutch Requirement'], inplace=True)
df_jobs['Dutch Requirement'] = np.where(
    df_jobs['Job Description'].str.contains(dutch_requirement_pattern),
    'Yes',
    'No',
)

# English
if 'English Requirement' in df_jobs.columns:
    df_jobs.drop(columns=['English Requirement'], inplace=True)
df_jobs['English Requirement'] = np.where(
    df_jobs['Job Description'].str.contains(english_requirement_pattern),
    'Yes',
    'No',
)

In [ ]:
df_jobs['Dutch Requirement'].value_counts()


In [ ]:
df_sectors = pd.read_pickle(f'{scraped_data}CBS/Data/Sectors Output from script.pkl').reset_index(drop=True)


In [ ]:
df_sectors.columns = [
    '_'.join(col) 
    if 'SBI Sector Titles' not in col 
    and 'Total Workforce' not in col 
    else col[-1] 
    for col in df_sectors.columns
]


In [ ]:
df_sectors.columns


In [ ]:
df_sectors.rename(
    columns={
        'Keywords': 'Search Keyword', 
        'Code': 'Sector Code', 
        'Sector Name': 'Sector', 
        'Gender_Female_n': 'Female Count (x 1000)', 
        'Gender_Male_n': 'Male Count (x 1000)', 
        'Gender_Sectoral Gender Segregation_Dominant Category': 'Gender', 
        'Age_Sectoral Age Segregation_Dominant Category': 'Age', 
        'n': 'Sector Count (x 1000)', 
    },
    inplace=True
)


In [ ]:
type(df_sectors['Search Keyword'][1])

In [ ]:
df_sectors = df_sectors.explode(
    'Search Keyword', ignore_index=True
).reset_index(drop=True)


In [ ]:
df_jobs = df_jobs.merge(df_sectors, on='Search Keyword', how='left')


In [ ]:
len(df_jobs)


In [ ]:
df_jobs.columns


In [ ]:
if df_jobs['Search Keyword'].loc[df_jobs['Sector'].isna().sum()] != 0:
    print('Some search keywords did not match a sector. Fixing')
    print(set(df_jobs['Search Keyword'].loc[df_jobs['Sector'].isna()].to_list()))
    print(len(df_jobs['Search Keyword'].loc[df_jobs['Search Keyword'].isin(list(keyword_trans_dict.keys()))]))
    df_jobs = fix_keywords(df_jobs)
    print(set(df_jobs['Search Keyword'].loc[df_jobs['Sector'].isna()].to_list()))
    print(len(df_jobs['Search Keyword'].loc[df_jobs['Search Keyword'].isin(list(keyword_trans_dict.keys()))]))


In [ ]:
len(df_jobs.loc[(df_jobs['Job Description'].str.contains(str_fix_incl)) | (df_jobs['Job Description'].str.contains(str_fix_eg))])


In [ ]:
# job_descriptions = list(set(df_jobs['Job Description'].to_list()))


In [ ]:
job_descriptions = list(
    set(
        df_jobs['Job Description'].loc[
            (df_jobs['Job Description'].str.contains(str_fix_incl)) |
            (df_jobs['Job Description'].str.contains(str_fix_eg))
        ].to_list()
    )
)


In [ ]:
len(job_descriptions)


In [ ]:
job_descriptions[0][1500:2000]

In [ ]:
df_jobs = df_jobs.loc[
    (df_jobs['Job Description'].str.contains(str_fix_incl)) | 
    (df_jobs['Job Description'].str.contains(str_fix_eg))
].reset_index(drop=True)

In [ ]:
len(df_jobs)

In [ ]:
df_jobs.columns

In [ ]:
# Function to make a list of punctuations that determine sentence boundry, i.e., split characters
def make_custom_punct_chars(main_punct_chars = [':', '|'], repeated_punct_chars = ['\n', ',']):
    custom_punct_chars = []
    temp_multi = []
    temp_spaced = []

    for punct_char in main_punct_chars:
        custom_punct_chars+= f'{punct_char}', f'{punct_char} '

    for idx in range(4):
        for punct_char in repeated_punct_chars:
            temp_multi.append(f'{punct_char}'*int(idx+1))
            temp_spaced.append(f'{punct_char} '*int(idx+1))

    for multi, spaced in zip(temp_multi, temp_spaced):
        custom_punct_chars+= multi, spaced

    custom_punct_chars.remove(',')
    custom_punct_chars.remove(', ')

    return custom_punct_chars


In [ ]:
custom_punct_chars = make_custom_punct_chars()

In [ ]:
custom_punct_chars

In [ ]:
# Set up Spacy
import spacy
from spacy.symbols import NORM, ORTH, LEMMA, POS

nlp = spacy.load('en_core_web_sm')


In [ ]:
# Add sentencizer to nlp pip and set custom punctuations
sentencizer = nlp.add_pipe('sentencizer')
sentencizer.punct_chars.update(custom_punct_chars)

# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     with open(f'{data_dir}punctuations.txt', 'wb') as f:
#         pickle.dump(sentencizer.punct_chars, f)

# with open(f'{data_dir}punctuations.txt', 'rb') as f:
#     custom_punct_chars = pickle.load(f)


In [ ]:
# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     with open(f'{data_dir}punctuations.txt', 'wb') as f:
#         pickle.dump(sentencizer.punct_chars, f)

In [ ]:
# with open(f'{data_dir}punctuations.txt', 'rb') as f:
#     custom_punct_char = pickle.load(f)

In [ ]:
from spacy.symbols import NORM, ORTH, LEMMA

special_cases_dict = {
    'incl.': [{65: 'incl', 67: 'including'}],
    'incl. ': [{65: 'incl', 67: 'including'}],
    '(incl.': [{65: 'incl', 67: 'including'}],
    'etc.': [{65: 'etc', 67: 'et cetera'}],
    'etc. ': [{65: 'etc', 67: 'et cetera'}],
    'e.g.': [{65: 'e.g', 67: 'for example'}],
    'e.g. ': [{65: 'e.g', 67: 'for example'}],
}

nlp.tokenizer.rules.update(special_cases_dict)


In [ ]:
nlp.tokenizer.rules['e.g.']

In [ ]:
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    df_jobs['Job Description spacy_sentencized'] = df_jobs['Job Description'].apply(
        lambda job_description: [
            sent 
            for sentence in nlp(job_description).sents 
            for sent in re.split(pattern, sentence.text) 
            if len(sent) != 0 
        ]
    )


In [ ]:
df_jobs['Job Description spacy_sentencized']


In [ ]:
df_jobs = df_jobs.explode('Job Description spacy_sentencized', ignore_index=True).reset_index(drop=True)


In [ ]:
df_jobs['Job Description spacy_sentencized_lower'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: job_sentence.strip().lower()
)


In [ ]:
df_jobs.head()

In [ ]:
len(df_jobs)

In [ ]:
# Spacy tokenize
df_jobs['Job Description spacy_tokenized'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: [
        str(token.text.strip().lower()) 
        for token in nlp.tokenizer(job_sentence) 
        if len(token) != 0 
        and not token.is_stop 
        and not token.is_punct 
        and not token.text in custom_punct_chars
    ]
)


In [ ]:
df_jobs['Job Description spacy_tokenized']

In [ ]:
# Load NLK
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer, LancasterStemmer

nltk_path = f'{llm_path}/nltk'
nltk.data.path.append(nltk_path)

nltk.download('words', download_dir = nltk_path)
nltk.download('stopwords', download_dir = nltk_path)
nltk.download('punkt', download_dir = nltk_path)
nltk.download('averaged_perceptron_tagger', download_dir = nltk_path)

stop_words = set(stopwords.words('english'))
punctuations = list(string.punctuation)


In [ ]:
# pattern = r'[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?<=[a-z])(?=[A-Z])'
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'

pattern_numbers = r'[^a-zA-Z0-9]'

In [ ]:
# nltk_sentencized = []

# for job_description in job_descriptions:
#     for sentence in sent_tokenize(job_description):
#         for sent in re.split(pattern, sentence):
#             nltk_sentencized.append(sent)

In [ ]:
nltk_sentencized = [
    sent 
    for job_description in job_descriptions 
    for sentence in sent_tokenize(job_description) 
    for sent in re.split(pattern, sentence)
    if len(sent) != 0
]

In [ ]:
len(nltk_sentencized)

In [ ]:
# df_jobs['Job Description nltk_sentencized'] = df_jobs['Job Description'].apply(
#     lambda job_description: [
#         sent 
#         for sentence in sent_tokenize(job_description) 
#         for sent in re.split(pattern, sentence)
#         if len(sent) != 0
#     ]
# )

In [ ]:
# df_jobs['Job Description nltk_sentencized'] = df_jobs['Job Description'].apply(
#     lambda job_description: [
#         re.split(pattern, sentence)
#         for sentence in sent_tokenize(job_description)
#         if len(sentence) != 0
#     ]
# )

In [ ]:
# df_jobs['Job Description nltk_sentencized'][0][-10:-1]

In [ ]:
# len(df_jobs['Job Description nltk_sentencized'][0]) + len(df_jobs['Job Description nltk_sentencized'][1])


In [ ]:
# df_jobs = df_jobs.explode('Job Description nltk_sentencized', ignore_index=True).reset_index(drop=True)

In [ ]:
# len(df_jobs)

In [ ]:
# df_jobs['Job Description nltk_sentencized']

In [ ]:
nltk_sentencized_lower = [
    str(sent.strip().lower()) 
    for sent in nltk_sentencized
]

In [ ]:
len(nltk_sentencized_lower)

In [ ]:
# df_jobs['Job Description nltk_sentencized_lower'] = df_jobs['Job Description nltk_sentencized'].apply(
#     lambda job_sentence: job_sentence.strip().lower()
# )


In [ ]:
# df_jobs['Job Description nltk_sentencized_lower']


In [ ]:
for index, sentence in df_jobs['Job Description spacy_sentencized'].items():
    if str_fix_eg in sentence:
        print(index)


In [ ]:
df_jobs['Job Description spacy_sentencized'][17:25].to_list()

In [ ]:
for index, sentence in df_jobs['Job Description spacy_sentencized'].items():
    if 'Power' in sentence:
        print(index)

In [ ]:
df_jobs['Job Description spacy_sentencized'][60:70].to_list()

In [ ]:
for idx, sent in enumerate(nltk_sentencized):
    if str_fix_eg in sent:
        print(idx)

In [ ]:
next(idx for idx, sent in enumerate(nltk_sentencized) if str_fix_eg in sent)

In [ ]:
# str_fix = 'Apply appropriate and effective communication methods to senior management and important stakeholders \(incl.'

for idx, sent in enumerate(nltk_sentencized):
    if str_fix_incl.split('\(')[0] in sent:
        print(idx)

In [ ]:
nltk_sentencized[84:95]

In [ ]:
# sentences split on e.g. and incl.
nltk_sentencized[
    next(idx for idx, sent in enumerate(nltk_sentencized) if str_fix_eg in sent):next(idx for idx, sent in enumerate(nltk_sentencized) if str_fix_incl.split('\(')[0] in sent)+2
]


In [ ]:
for idx, sent in enumerate(nltk_sentencized):
    if 'Power' in sent:
        print(idx)

In [ ]:
next(idx for idx, sent in enumerate(nltk_sentencized) if 'Power' in sent)

In [ ]:
nltk_sentencized[next(idx for idx, sent in enumerate(nltk_sentencized) if 'Power' in sent)]

In [ ]:
for idx, sentence in enumerate(nltk_sentencized[84:95]):
    print(f'Sentence {idx+1}: {sentence}\n')

In [ ]:
# nltk_tokenized = []

# for job_sentence in nltk_sentencized:
#     for token in word_tokenize(job_sentence):
#         if len(token) != 0 and token != '...' and token.lower() not in set(stopwords.words('english')) and token.lower() not in list(string.punctuation):
#             nltk_tokenized.append(str(token.strip().lower())))


In [ ]:
nltk_tokenized = [
    str(token.strip().lower()) 
    for job_sentence in nltk_sentencized 
    for token in word_tokenize(job_sentence) 
    if len(token) != 0 
    and token != '...' 
    and not token.lower() in set(stopwords.words('english')) 
    and not token.lower() in list(string.punctuation) 
]

In [ ]:
df_jobs['Job Description nltk_tokenized'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: [
        str(token.strip().lower()) 
        for token in word_tokenize(job_sentence) 
        if len(token) != 0 
        and token != '...' 
        and not token.lower() in set(stopwords.words('english')) 
        and not token.lower() in list(string.punctuation) 
    ]
)


In [ ]:
df_jobs['Job Description nltk_tokenized']

In [ ]:
'...' in nltk_tokenized

In [ ]:
len(nltk_tokenized)

In [ ]:
from gensim.utils import save_as_line_sentence, simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string, preprocess_documents

gensim_tokenized = []

for job_sentence in nltk_sentencized:
    gensim_tokenized.append(preprocess_string(re.sub(pattern_numbers, ' ', job_sentence.strip().lower())))

In [ ]:
gensim_tokenized[0]

In [ ]:
df_jobs['Job Description gensim_tokenized'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: preprocess_string(re.sub(pattern_numbers, ' ', job_sentence.strip().lower()))
)


In [ ]:
df_jobs['Job Description gensim_tokenized']

In [ ]:
nltk_token_tags = pos_tag(nltk_tokenized)

In [ ]:
nltk_token_tags[1][1]

In [ ]:
df_jobs['Job Description nltk_token_tags'] = df_jobs['Job Description nltk_tokenized'].apply(
    lambda token: pos_tag(token)
)


In [ ]:
df_jobs['Job Description nltk_token_tags']

In [ ]:
# https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

In [ ]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([token])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


In [ ]:
import unicodedata

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

nltk_lemmas = []
nltk_stems = []

for token in nltk_tokenized:
    token = unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    token_lemma = lemmatizer.lemmatize(token)
    token_stem = stemmer.stem(token)
    token_pos = get_wordnet_pos(token)
    token_pos_lemma = lemmatizer.lemmatize(token, token_pos)
    print(f'Token: {token} | Stem: {token_stem} | Lemma: {token_lemma} | POS Lemma: {token_pos_lemma}')
    nltk_lemmas.append(token_pos_lemma)
    nltk_stems.append(token_stem)


In [ ]:
import unicodedata

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

nltk_lemmas = []
nltk_stems = []

for token in nltk_tokenized:
    token = unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    token_lemma = lemmatizer.lemmatize(token)
    token_stem = stemmer.stem(token)
    token_pos = get_wordnet_pos(token)
    nltk_lemmas.append(lemmatizer.lemmatize(token, get_wordnet_pos(token)))
    nltk_stems.append(token_stem)


In [ ]:
nltk_lemmas = [
    lemmatizer.lemmatize(
        token, get_wordnet_pos(
            unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        )
    )
    for token in nltk_tokenized
]

In [ ]:
df_jobs['Job Description nltk_lemmas'] = df_jobs['Job Description spacy_tokenized'].apply(
    lambda tokens: [
        lemmatizer.lemmatize(
            token, get_wordnet_pos(
                unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            )
        )
        for token in tokens
    ]
)


In [ ]:
df_jobs['Job Description nltk_lemmas']

In [ ]:
len(nltk_lemmas)

In [ ]:
import unicodedata

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

nltk_lemmas = []
nltk_stems = []

for token in nltk_tokenized:
    token = unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    nltk_stems.append(stemmer.stem(unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')))


In [ ]:
nltk_stems = [
    stemmer.stem(
        unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    )
    for token in nltk_tokenized
]

In [ ]:
df_jobs['Job Description nltk_stems'] = df_jobs['Job Description spacy_tokenized'].apply(
    lambda tokens: [
        stemmer.stem(
            unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        )
        for token in tokens
    ]
)


In [ ]:
df_jobs['Job Description nltk_stems']

In [ ]:
len(nltk_stems)

In [ ]:
df_jobs.columns

In [ ]:
# special_cases = {":)": [{"ORTH": ":)"}]}
# prefix_re = re.compile(r'''^[\\[\\("']''')
# suffix_re = re.compile(r'''[\\]\\)"']$''')
# infix_re = re.compile(r'''[-~]''')
# simple_url_re = re.compile(r'''^https?://''')

# def custom_tokenizer(nlp):
#     return Tokenizer(nlp.vocab, rules=special_cases,
#                                 prefix_search=prefix_re.search,
#                                 suffix_search=suffix_re.search,
#                                 infix_finditer=infix_re.finditer,
#                                 url_match=simple_url_re.match)

In [ ]:
# nlp.tokenizer.add_special_case('incl', [{ORTH: u'incl', NORM: u'include', LEMMA: u'include', POS: u'VERB'}])


In [ ]:
# # Spacy sentencizer
# spacy_sentencized = []

# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     for job_description in job_descriptions:
#         for sentence in nlp(job_description).sents:
#             for sent in re.split(pattern, sentence.text):
#                 if len(sent) != 0:
#                     spacy_sentencized.append(sent)

In [ ]:
# Spacy sentencizer
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    spacy_sentencized = [
        sent 
        for job_description in job_descriptions 
        for sentence in nlp(job_description).sents 
        for sent in re.split(pattern, sentence.text) 
        if len(sent) != 0 
    ]
    
    spacy_sentencized_lower = [
        str(sent.strip().lower()) 
        for sent in spacy_sentencized
    ]


In [ ]:
# # Spacy sentencizer
# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     spacy_sentencized = [
#         re.split(pattern, sentence.text) 
#         for job_description in job_descriptions 
#         for sentence in nlp(job_description).sents 
# #         for sent in re.split(pattern, sentence.text) 
#         if len(sentence) != 0 
#     ]



In [ ]:
spacy_sentencized[23]

In [ ]:
# Spacy tokenizer
spacy_tokenized = []

if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    for job_sentence in spacy_sentencized:
        for token in nlp.tokenizer(job_sentence):
            if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars:
                spacy_tokenized.append(token.text)

In [ ]:
spacy_tokenized

In [ ]:
type(spacy_sentencized[0])

In [ ]:
len(spacy_sentencized)

In [ ]:
for idx, sent in enumerate(spacy_sentencized):
    if str_fix_eg in sent:
        print(idx)

In [ ]:
next(idx for idx, sent in enumerate(spacy_sentencized) if str_fix_eg in sent)

In [ ]:
# str_fix = 'Apply appropriate and effective communication methods to senior management and important stakeholders \(incl.'


for idx, sent in enumerate(spacy_sentencized):
    if str_fix_incl.split('\(')[0] in sent:
        print(idx)

In [ ]:
next(idx for idx, sent in enumerate(spacy_sentencized) if str_fix_incl.split('\(')[0] in sent)

In [ ]:
spacy_sentencized[18:25]

In [ ]:
for idx, sent in enumerate(spacy_sentencized):
    if 'Power' in sent:
        print(idx)

In [ ]:
next(idx for idx, sent in enumerate(spacy_sentencized) if 'Power' in sent)

In [ ]:
spacy_sentencized[63]

In [ ]:
for idx, sentence in enumerate(spacy_sentencized[18:25]):
    print(f'Sentence {idx+1}: {sentence}\n')

In [ ]:
# # Spacy tokenizer
# spacy_tokenized = []

# for job_sentence in spacy_sentencized:
# #         doc = nlp.tokenizer(job_sentence)
#     spacy_tokenized.extend(
#         [
#             token.text for token in nlp.tokenizer(job_sentence) 
#             if token.text not in custom_punct_chars
#             and not token.is_stop 

#         ]
#     )

In [ ]:
type(spacy_sentencized[0])

In [ ]:
spacy_sentencized[6]

In [ ]:
# Spacy tokenizer

spacy_tokenized = [
    str(token.text.strip().lower()) 
    for job_sentence in spacy_sentencized 
    for token in nlp.tokenizer(job_sentence) 
    if len(token) != 0 
    and not token.is_stop 
    and not token.is_punct 
    and not token.text in custom_punct_chars
]


In [ ]:
len(spacy_tokenized)

In [ ]:
spacy_tokenized[:10]

In [ ]:
# nlp.get_pipe('attribute_ruler').add([[{"TEXT":"Angeltown"}]],{"LEMMA":"San Fransisco"})

In [ ]:
# # Spacy pos tagging
# nlp_stemmer = LancasterStemmer()
nlp_stemmer = PorterStemmer()

nlp_token_tags = []
nlp_lemmas = []
nlp_stems = []

for job_description in job_descriptions:
    for token in nlp(job_description):
        if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars:
            nlp_token_tags.append(tuple([token.text, token.tag_]))
            nlp_lemmas.append(token.lemma_)
            nlp_stems.append(nlp_stemmer.stem(token.text))
#         for sentence in doc.sents:
#             for sent in re.split(pattern, sentence.text):
#                 if len(sent) != 0:
#                     spacy_sentencized.append(sent)

In [ ]:
nlp_token_tags = [
    tuple([token.text.strip().lower(), token.tag_])
    for job_description in job_descriptions
    for token in nlp(job_description)
    if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars
]


In [ ]:
df_jobs['Job Description nlp_token_tags'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: [
        tuple([token.text.strip().lower(), token.tag_])
        for token in nlp(job_sentence)
        
    ]
)


In [ ]:
nlp_lemmas = [
    token.lemma_.strip().lower()
    for job_description in job_descriptions
    for token in nlp(job_description)
    if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars
]

In [ ]:
df_jobs['Job Description nlp_lemmas'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: [
        token.lemma_.strip().lower()
        for token in nlp(job_sentence)
        if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars
    ]
)


In [ ]:
nlp_stems = [
    stemmer.stem(token.text.strip().lower())
    for job_description in job_descriptions
    for token in nlp(job_description)
    if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars
]

In [ ]:
df_jobs['Job Description nlp_stems'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: [
        stemmer.stem(token.text.strip().lower())
        for token in nlp(job_sentence)
        if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars
    ]
)


In [ ]:
df_jobs['Job Description nlp_stems']

In [ ]:
# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     nlp_token_tags = [
#         tuple([token.text, token.tag_]) 
#         for job_description in job_descriptions 
#         for token in nlp(job_description) 
#         if len(token) != 0 
#         and not token.is_stop 
#         and not token.is_punct 
#         and not token.text in custom_punct_chars
#     ]

In [ ]:
def get_word_num_and_frequency(row, text_col):

    row['Job Description num_words'] = len(str(row[f'{text_col}']).split())
    row['Job Description num_unique_words'] = len(set(str(row[f'{text_col}']).split()))
    row['Job Description num_chars'] = len(str(row[f'{text_col}']))
    row['Job Description num_punctuations'] = len([c for c in str(row[f'{text_col}']) if c in string.punctuation])

    return row


In [ ]:
df_jobs = df_jobs.apply(lambda row: get_word_num_and_frequency(row=row, text_col='Job Description spacy_sentencized'), axis='columns')

In [ ]:
# import pandas as pd
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_tags_lemmas_stems_spacy_nltk.pkl').reset_index(drop=True)


In [ ]:
df_jobs = df_jobs[0:2]

In [ ]:
df_jobs['Job Description bert_tokenized']

In [ ]:
x = df_jobs['Job Description spacy_tokenized'].str.join(' ').to_list()

In [ ]:
x = df_jobs['Job Description bert_tokenized'].to_list()

In [ ]:
x

In [ ]:
sentence

In [ ]:
x = df_jobs['Job Description bert_tokenized'].to_list()[1]

In [ ]:
x[0]

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline, BertTokenizer, BertForPreTraining, BertConfig, BertModel
bert_model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name, strip_accents = True)
bert_model = BertModel.from_pretrained(bert_model_name)



In [ ]:
sentence = df_jobs['Job Description spacy_sentencized'].to_list()[1]


u = bert_tokenizer.tokenize(sentence)

In [ ]:
bert_tokenizer.cls_token

In [ ]:
u

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline, BertTokenizer, BertForPreTraining, BertConfig, BertModel
bert_model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name, strip_accents = True)
bert_model = BertModel.from_pretrained(bert_model_name)

# bert_pos_model_name = 'QCRI/bert-base-multilingual-cased-pos-english'
# bert_pos_tokenizer = AutoTokenizer.from_pretrained(bert_pos_model_name)
bert_pos_model = AutoModelForTokenClassification.from_pretrained('QCRI/bert-base-multilingual-cased-pos-english')
bert_pos_tag = TokenClassificationPipeline(model=bert_pos_model, tokenizer=bert_tokenizer)


In [ ]:

tokens = df_jobs['Job Description spacy_tokenized'].to_list()
# x = bert_pos_tag(sentence)

In [ ]:
tokens

In [ ]:
u=df_jobs['Job Description spacy_tokenized'].apply(
    lambda tokens: bert_pos_tag(token)[i] for token in tokens for i in range(len(token))
)

In [ ]:
i=0
# for token in tokens:
#     for i in range(len(tokens)):
#         print(i)
print(bert_pos_tag(token)[i] for i in range(len(token)) for token in tokens)

In [ ]:
for token in tokens:
    print(bert_pos_tag(token))
#     for i in range(len(token)):
#         for pos_data in bert_pos_tag(token)[i]:
#             print(token[i], pos_data['word'], pos_data['entity'])

In [ ]:
i = [
    (pos_data['word'], pos_data['entity'])
    for token in tokens
    for i in range(len(token))
    for pos_data in bert_pos_tag(token)[i]
]


In [ ]:
i

In [ ]:
bert_pos_model_name = 'QCRI/bert-base-multilingual-cased-pos-english'
bert_pos_model = AutoModelForTokenClassification.from_pretrained(bert_pos_model_name)
bert_pos_tag = TokenClassificationPipeline(model=bert_pos_model, tokenizer=bert_tokenizer)

df_jobs['Job Description bert_token_tags'] = df_jobs['Job Description bert_tokenized'].apply(
    lambda tokens: [
        (pos_data['word'], pos_data['entity'])
        for i in range(len(tokens))
        for pos_data in bert_pos_tag(tokens)[i]
    ]
)


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
text = "I saw The Who perform. Who did you see?"
doc1 = nlp(text)
print(doc1[2], doc1[2].tag_, doc1[2].pos_)  # DT DET
print(doc1[3], doc1[3].tag_, doc1[3].pos_)  # WP PRON

In [ ]:
text = 'I saw The Who "perform". Who did you see?'

In [ ]:
for token in nlp(text):
    print(token.is_punct)

In [ ]:
doc1.

In [ ]:
for token in doc1:
    print(token.is_alpha)

In [ ]:
for tokens in sentence:
    for token in tokens:
        print(bert_pos_tag(token)[0])
        
#     for idx, token in enumerate(tokens):
#         print(tokens[idx],bert_pos_tag(token)[0])

In [ ]:
s

In [ ]:
for token in sentence:
    print(token)

In [ ]:
i = df_jobs['Job Description spacy_sentencized'].apply(
    lambda sentence: bert_pos_tag(sentence)
)

In [ ]:
for idx, tokens in df_jobs['Job Description spacy_tokenized'].items():
    print(bert_pos_tag(tokens)[idx])

In [ ]:
j = df_jobs['Job Description spacy_tokenized'].apply(
    lambda tokens: bert_pos_tag(token)[idx]
    for idx, token in enumerate(tokens)
)

In [ ]:
j.loc[0, '<lambda>'].to_list()

In [ ]:
# j[1]

In [ ]:
# i[1]

In [ ]:
df_jobs['Job Description spacy_tokenized'][1]

In [ ]:
j

In [ ]:
[(bert_pos_tag(sentence)['word'], bert_pos_tag(sentence)(['entit)y'])]

In [ ]:
outputs[0][0]['entity']

In [ ]:
outputs[0][0]['word']

In [ ]:
[(bert_tags['word'], outpbert_tagsuts['entity'])]

In [ ]:
sentence[outputs[0][0]['start']]

In [ ]:
# for sent in spacy_sentencized:
#     num_words = len(sent.split())
#     num_unique_words = len(set(sent.split()))
#     num_chars = len(sent)
#     num_punctuations = len([c for c in sent if c in string.punctuation])

In [ ]:
# Sentiment
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentim_analyzer = SentimentIntensityAnalyzer()

def get_sentiment(df_jobs_to_be_processed, text_col, algo='vader', sentiment_range=(-1,1)):

    ## calculate sentiment
    if algo == 'vader':
        df_jobs_to_be_processed['Sentiment'] = df_jobs_to_be_processed[text_col].apply(lambda x: sentim_analyzer.polarity_scores(x)['compound'] if isinstance(x, str) else np.nan)
    elif algo == 'textblob':
        df_jobs_to_be_processed['Sentiment'] = df_jobs_to_be_processed[text_col].apply(lambda x: TextBlob(x).sentiment.polarity)
    ## rescaled
    if sentiment_range != (-1,1):
        df_jobs_to_be_processed['Sentiment'] = preprocessing.MinMaxScaler(feature_range=sentiment_range).fit_transform(df_jobs_to_be_processed[['sentiment']])
    # print(df_jobs_to_be_processed[['sentiment']].describe().T)

    return df_jobs_to_be_processed


In [ ]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp.add_pipe('spacytextblob')

print()

In [ ]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

# nlp.add_pipe('spacytextblob')
sentiment = []
for job_sentence in spacy_sentencized:
    sentiment.append(nlp(job_sentence)._.blob.polarity)

In [ ]:
sentiment = [
    float(nlp(job_sentence)._.blob.polarity)
    for job_sentence in spacy_sentencized
]

In [ ]:
df_jobs['Job Description spacy_sentiment'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda sentence: float(nlp(sentence)._.blob.polarity)
)


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentim_analyzer = SentimentIntensityAnalyzer()

df_jobs['Job Description nltk_sentiment'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda sentence: float(sentim_analyzer.polarity_scores(sentence)['compound'])
    if isinstance(sentence, str) else np.nan
)


In [ ]:
df_jobs['Job Description nltk_sentiment']

In [ ]:
df_jobs = get_sentiment(df_jobs, text_col='Job Description spacy_sentencized')


In [ ]:
df_jobs['Sentiment']

In [ ]:
df_jobs.head()

In [ ]:
# Sentiment
from nltk.sentiment.vader import SentimentIntensityAnalyzer

vader = SentimentIntensityAnalyzer()

for sent in spacy_sentencized:
    print(vader.polarity_scores(sent)['compound'])


In [ ]:
# # Sentiment
# from textblob import TextBlob, Word

# for sent in spacy_sentencized:
#     print(TextBlob(sent).sentiment.polarity)


In [ ]:
# Bert Tokenizer
from transformers import BertTokenizer, AutoTokenizer, BertForPreTraining, BertConfig, BertModel

bert_model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name, strip_accents = True)
bert_model = BertModel.from_pretrained(bert_model_name)

df_jobs['bert_tokenized'] = df_jobs['Job Description spacy_sentencized'].apply(lambda sentence: bert_tokenizer.tokenize(str(sentence)))



In [ ]:
df_jobs['bert_tokenized']

In [ ]:
from gensim.models.phrases import ENGLISH_CONNECTOR_WORDS, Phraser, Phrases

# Bigrams
bigram = Phraser(Phrases(df_jobs['Job Description spacy_tokenized'], connector_words=ENGLISH_CONNECTOR_WORDS, min_count=1, threshold=1))
df_jobs['Job Description gensim_2garms'] = bigram[df_jobs['Job Description spacy_tokenized']]

# Trigrams
trigram = Phraser(Phrases(bigram_sentences, connector_words=ENGLISH_CONNECTOR_WORDS, min_count=1, threshold=1))
df_jobs['Job Description gensim_3garms'] = trigram[bigram_sentences]

In [ ]:
df_jobs['Job Description gensim_3garms']

In [ ]:
for idx, row in df_jobs['Job Description gensim_3garms'].items():
    for token in row:
        if '_' in token:
            if re.search('_([^_]+)_', token):
                print(token)

In [ ]:
spacy_sentences = ''
for token in spacy_tokenized:
    spacy_sentences+= token + ' '

In [ ]:
spacy_sentences = [spacy_sentences.strip().lower()]

In [ ]:
spacy_bigrams = []

for job_sentence in spacy_sentences:
#     print(job_sentence)
    for noun_phrase in list(nlp(job_sentence).noun_chunks):
        print(noun_phrase.merge_noun_chunks)
#         print(noun_phrase.merge(noun_phrase.root.tag_, noun_phrase.root.lemma_, noun_phrase.root.ent_type_))
#         if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars:
#             spacy_tokenized.append(token.text)

In [ ]:
def huggingface_similarity(sentences, bert_model):

    embeddings = bert_model.encode(sentences, show_progress_bar=True)
    cos_sim = util.cos_sim(embeddings, embeddings)

    #Sort list by the highest cosine similarity score
    all_sentence_combinations = sorted([[cos_sim[i][j], i, j] for i in range(len(cos_sim)-1) for j in range(i+1, len(cos_sim))], key=lambda x: x[0], reverse=True)

    if args['print_enabled'] is True:
        print('Top-5 most similar pairs:')
        for score, i, j in all_sentence_combinations[:5]:
            print(f'{sentences[i]} \t {sentences[j]} \t {cos_sim[i][j]:.4f}')

    if args['save_enabled'] is True:
        bert_model.to_json_file('bert_config.json')

    return all_sentence_combinations, embeddings

In [ ]:
for sentences in spacy_sentencized:
    print(bert_tokenizer.encode_plus(sentences))

In [ ]:
from transformers import AutoTokenizer, BertModel
import torch

tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
model = BertModel.from_pretrained(bert_model_name)

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

In [ ]:
# Set up Bert
from transformers import AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline, BertTokenizer, BertForPreTraining, BertConfig, BertModel
bert_model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name, strip_accents = True)
bert_model = BertModel.from_pretrained(bert_model_name)


In [ ]:
bert_tokenizer.tokenize

In [ ]:
last_hidden_states

In [ ]:
cls(
    BertModel.from_pretrained(
        bert_model_name,
        output_attentions=True,
        output_hidden_states=True,
        output_additional_info=True,
    ),
    BertAligner.from_pretrained(bert_model_name),
)

In [ ]:
# Spacy sentencizer
# spacy_sentencized = []

if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    for job_description in job_descriptions:
        for token in nlp(job_description):
            print(token.text, token.tag_)
#             for sent in re.split(pattern, sentence.text):
#                 if len(sent) != 0:
#                     spacy_sentencized.append(sent)

In [272]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
languages = ["en", "['nl', 'en']", ['en', 'nl']]
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
nan_list = [None, 'None', '', ' ', [], -1, '-1', 0, '0', 'nan', np.nan, 'Nan']
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'
import string
import re
import time
import json
import csv
import glob
import pickle
import unicodedata
import pandas as pd
import numpy as np
import googletrans
from googletrans import Translator


from spacy.pipeline import Sentencizer

# Set up Spacy
import spacy
from spacy.symbols import NORM, ORTH, LEMMA, POS
from spacytextblob.spacytextblob import SpacyTextBlob
from spacy.matcher import Matcher

nlp = spacy.load('en_core_web_sm')

# Set up NLK
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer, LancasterStemmer
from nltk.tag import pos_tag, pos_tag_sents
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk_path = f'{llm_path}/nltk'
nltk.data.path.append(nltk_path)

nltk.download('words', download_dir = nltk_path)
nltk.download('stopwords', download_dir = nltk_path)
nltk.download('punkt', download_dir = nltk_path)
nltk.download('averaged_perceptron_tagger', download_dir = nltk_path)

stop_words = set(stopwords.words('english'))
punctuations = list(string.punctuation)
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
sentim_analyzer = SentimentIntensityAnalyzer()

# Set up Gensim
from gensim.utils import save_as_line_sentence, simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string, preprocess_documents
from gensim.models.phrases import ENGLISH_CONNECTOR_WORDS, Phraser, Phrases


[nltk_data] Downloading package words to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package averaged_perceptron_

In [ ]:
print('yes')

In [ ]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_tags_lemmas_stems_spacy_nltk.pkl').reset_index(drop=True)[:2]


In [ ]:
sentences = df_jobs['Job Description spacy_sentencized'].to_list()

In [ ]:
len(sentences)

In [ ]:
sentences

In [ ]:
from spacy.matcher import Matcher
nlp = spacy.load('en_core_web_sm')

matcher = Matcher(nlp.vocab)

# Bigrams
bigram_rules = [
    ['NOUN', 'VERB'],
    ['VERB', 'NOUN'],
    ['ADJ', 'NOUN'],
    ['ADJ', 'PROPN'],
    # more rules here...
]

trigram_rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

bigram_patterns = [[{'POS': i} for i in j] for j in bigram_rules]
trigram_patterns = [[{'POS': i} for i in j] for j in trigram_rules]


# rules = [
#     ['VERB', 'ADJ', 'NOUN'],
#     ['NOUN', 'VERB', 'ADV'],
#     ['NOUN', 'ADP', 'NOUN'],
#     # more rules here...
# ]

# lst1 = []

# for j in trigram_rules:
#     lst2 = []
#     for i in j:
# #         x = {'POS': i}
#         lst2.append({'POS': i})
# #         print(lst2)
#     lst1.append(lst2)
# #         lst2.append([{"POS": i} ])
# print(lst1)
# # trigram_patterns = [[{'POS': i} for i in j] for j in patterns]


In [ ]:
# lst1 = []

# for j in bigram_rules:
#     lst2 = []
#     for i in j:
# #         x = {'POS': i}
#         lst2.append({'POS': i})
# #         print(lst2)
#     lst1.append(lst2)
# #         lst2.append([{"POS": i} ])
# print(lst1)

In [ ]:
bigram_patterns = [[{'POS': i} for i in j] for j in bigram_rules]
trigram_patterns = [[{'POS': i} for i in j] for j in trigram_rules]


In [ ]:
trigram_patterns

In [ ]:
# trigram_patterns = {
#     'noun_verb': [{'POS': 'NOUN'}, {'POS': 'VERB'}],
#     'verb_noun': [{'POS': 'VERB'}, {'POS': 'NOUN'}],
#     'adj_noun': [{'POS': 'ADJ'}, {'POS': 'NOUN'}],
#     'adj_propn': [{'POS': 'ADJ'}, {'POS': 'PROPN'}],
# }

In [ ]:
matcher = Matcher(nlp.vocab)

matcher.add('bigram_patterns', bigram_patterns)
matcher.add('trigram_patterns', trigram_patterns)


In [ ]:
sentences[4]

In [ ]:
bigram = []

for sentence in sentences:
    for match_id, start, end in matcher(nlp(sentence)):
#         sent = nlp(sentence)
        bigram.append(nlp(sentence)[start:end].text)

In [ ]:
bigram


In [ ]:
bigram = [
    nlp(sentence)[start:end].text
    for sentence in sentences
    for match_id, start, end in matcher(nlp(sentence))
]

In [4]:
print('yes')

yes


In [ ]:
lst = []
gram_type = 'bigram'
for sentence in sentences:
    doc = nlp(sentence)
    for match_id, start, end in matcher(doc):
        if nlp.vocab.strings[match_id].split('_')[0] == gram_type:
            matches = doc[start:end].text
            print(f"{doc[:start]} {'_'.join(doc[start:end].text.split())} {doc[end:]}")
#     lst.append(x)

In [ ]:
lst

In [ ]:
df_jobs['Job Description spacy_sentencized']

In [ ]:
for idx, tokens in df_jobs['Job Description bert_tokenized'].items():
    for i in range(len(tokens)):
        for pos_data in bert_pos_tag(tokens)[i]:
            print(pos_data['word'])

In [ ]:
# Set up Bert
from transformers import AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline, BertTokenizer, BertForPreTraining, BertConfig, BertModel
bert_model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name, strip_accents = True)
bert_model = BertModel.from_pretrained(bert_model_name)

bert_pos_model_name = 'QCRI/bert-base-multilingual-cased-pos-english'
bert_pos_model = AutoModelForTokenClassification.from_pretrained(bert_pos_model_name)
bert_pos_tag = TokenClassificationPipeline(model=bert_pos_model, tokenizer=bert_tokenizer)


In [ ]:

df_jobs['Job Description bert_token_tags_with_scores'] = df_jobs['Job Description bert_tokenized'].apply(
    lambda tokens: [
        (pos_data['word'], pos_data['entity'], pos_data['score'])
        for i in range(len(tokens))
        for pos_data in bert_pos_tag(tokens)[i]
    ]
)


In [ ]:
df_jobs['Job Description bert_token_tags_with_scores']

In [ ]:
# df_jobs['Job Description bert_token_tags_with_scores'].apply(
#     lambda tag_list: print(tag_tuple[i])
#     for i, tag_tuple in enumerate(tag_list)
# )

In [ ]:
for i in range(len(tokens))

In [ ]:
def tags_drop_scores(tag_list):

    return [
        [(tag_list[i][0], tag_list[i][1])]
        for tag_tuple in tag_list
        for i in range(len(tag_list))
    ]


In [ ]:
df_jobs['Job Description bert_token_tags'] = df_jobs['Job Description bert_token_tags_with_scores'].apply(
    lambda tag_list: tags_drop_scores(tag_list)
)


In [ ]:
df_jobs['Job Description bert_token_tags']

In [273]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_tags_lemmas_stems_spacy_nltk.pkl').reset_index(drop=True)[:10]


In [274]:
print('yes')

yes


In [82]:
df_jobs['Job Description spacy_sentencized'][2]

'Its purpose is Enabling people and organisations to use our banking services by ensuring a compliant and competitive KYC ecosystem.'

In [8]:
sentences = df_jobs['Job Description spacy_sentencized'].to_list()

In [9]:
sentences[2]

'Its purpose is Enabling people and organisations to use our banking services by ensuring a compliant and competitive KYC ecosystem.'

In [10]:
# Spacy Bigrams
matcher = Matcher(nlp.vocab)

bigram_rules = [
    ['NOUN', 'VERB'],
    ['VERB', 'NOUN'],
    ['ADJ', 'NOUN'],
    ['ADJ', 'PROPN'],
    # more rules here...
]

bigram_patterns = [[{'POS': i} for i in j] for j in bigram_rules]

matcher.add('bigram_patterns', bigram_patterns)


In [ ]:
# def spacy_matches_ngrams(sentence, matcher, gram_type):
    
#     doc = nlp(sentence)
#     matches = matcher(doc)
#     matches_dict = {}

#     for match_id, start, end in matcher(doc):
#         if nlp.vocab.strings[match_id].split('_')[0] == gram_type:
#             match = doc[start:end].text
#             matches_dict[match] = '_'.join(match.split())
#             return matches_dict


In [239]:
def spacy_make_ngrams(sentence, matcher, gram_type):

    doc = nlp(sentence)
    matches = matcher(doc)
    matches_list = []

    for idx in range(len(matches)):
        for match_id, start, end in matches:
            if nlp.vocab.strings[match_id].split('_')[0] == gram_type:
                match = doc[matches[idx][1]: matches[idx][2]].text
                matches_list.append(match.lower())

    return [tuple(ngrams.split()) for ngrams in list(set(matches_list))]


In [240]:
# gram_type = 'bigram'

# def spacy_matches_ngrams(sentence, matcher, gram_type):

#     doc = nlp(sentence)
#     matches = matcher(doc)

#     matches_dict = {}

#     for idx, (match_id, start, end) in enumerate(matches):
#         print(matches[idx], doc[matches[idx][1]:matches[idx][2]].text)
#         print('-'*20)
# #         if nlp.vocab.strings[match_id].split('_')[0] == gram_type:
# #             matches_dict[doc[matches[idx][start]: matches[idx][end]].text] = matches[idx][start], matches[idx][end]
            
# #             print(match_id, start, end)
# #         matches_dict[doc[matches[idx][start]: matches[idx][end]].text] = matches[idx][start], matches[idx][end]
# # #     matches_list.append([matches_dict])
    
# #     return matches_dict
#     print(matches_dict)


In [241]:
df_jobs['Job Description spacy_1grams'] = df_jobs['Job Description spacy_tokenized']

In [204]:
for ngrams_list in df_jobs[f'Job Description spacy_2grams']:
    for ngram_tuple in ngrams_list:
        print('_'.join(ngram_tuple))
#             print(f'{ngram_tuple[i]} {ngram_tuple[i+1]}')


Enabling_people
competitive_KYC
department_providing
standardised_solutions
first_line
operational_excellence
related_regulations
many_colleagues
different_disciplines
global_organisation
other_lines
including_regulators
related_laws
content_expertise
sets_standards
various_KYC
external_parties
local_TM
efficient_compliance
related_regulations
vital_role
dedicated_teams


In [242]:
# Spacy bi and trigrams
matcher = Matcher(nlp.vocab)

bigram_rules = [
    ['NOUN', 'VERB'],
    ['VERB', 'NOUN'],
    ['ADJ', 'NOUN'],
    ['ADJ', 'PROPN'],
    # more rules here...
]

trigram_rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

patters_dict = {
    'bigram_patterns': [[{'POS': i} for i in j] for j in bigram_rules],
    'trigram_patterns': [[{'POS': i} for i in j] for j in trigram_rules],
}

ngram_dict = {
    'bigram': 2,
    'trigram': 3,
}

for ngram_name, ngram_num in ngram_dict.items():
    
    
    matcher.add(f'{ngram_name}_patterns', patters_dict[f'{ngram_name}_patterns'])

    df_jobs[f'Job Description spacy_{str(ngram_num)}grams'] = df_jobs['Job Description spacy_sentencized'].apply(
        lambda sentence:
        spacy_make_ngrams(sentence, matcher, ngram_name)
    )

    df_jobs[f'Job Description spacy_{str(ngram_num)}grams_in_sent'] = df_jobs['Job Description spacy_sentencized'].str.lower().replace(
        regex = {
            ' '.join(ngram_tuple): '_'.join(ngram_tuple)
            for ngrams_list in df_jobs[f'Job Description spacy_2grams']
            for ngram_tuple in ngrams_list
        }
    )
    
    if f'{ngram_name}_patterns' in matcher:
        matcher.remove(f'{ngram_name}_patterns')
    assert f'{ngram_name}_patterns' not in matcher

In [217]:
x = {
    ' '.join(ngram_tuple): '_'.join(ngram_tuple)
    for ngrams_list in df_jobs[f'Job Description spacy_2grams']
    for ngram_tuple in ngrams_list
}

print(x)

{'Enabling people': 'Enabling_people', 'competitive KYC': 'competitive_KYC', 'department providing': 'department_providing', 'standardised solutions': 'standardised_solutions', 'first line': 'first_line', 'operational excellence': 'operational_excellence', 'related regulations': 'related_regulations', 'many colleagues': 'many_colleagues', 'different disciplines': 'different_disciplines', 'global organisation': 'global_organisation', 'other lines': 'other_lines', 'including regulators': 'including_regulators', 'related laws': 'related_laws', 'content expertise': 'content_expertise', 'sets standards': 'sets_standards', 'various KYC': 'various_KYC', 'external parties': 'external_parties', 'local TM': 'local_TM', 'efficient compliance': 'efficient_compliance', 'vital role': 'vital_role', 'dedicated teams': 'dedicated_teams'}


In [216]:
x = {}
for ngrams_list in df_jobs[f'Job Description spacy_2grams']:
    for ngram_tuple in ngrams_list:
        print(' '.join(ngram_tuple))
#         print('_'.join(ngram_tuple))
#             print(f'{ngram_tuple[i]} {ngram_tuple[i+1]}')

Enabling people
competitive KYC
department providing
standardised solutions
first line
operational excellence
related regulations
many colleagues
different disciplines
global organisation
other lines
including regulators
related laws
content expertise
sets standards
various KYC
external parties
local TM
efficient compliance
related regulations
vital role
dedicated teams


In [ ]:
df_jobs[f'Job Description spacy_{str(ngram_num)}grams_in_sent'] = df_jobs['Job Description spacy_sentencized'].replace(
    regex = {
        ngram: '_'.join(ngram_tuple)
        for ngrams_list in df_jobs[f'Job Description spacy_2grams']
        for ngram_tuple in ngrams_list
    }
)

In [162]:
# df_jobs['Job Description spacy_bigram_lists'] = df_jobs['Job Description spacy_sentencized'].apply(
#     lambda sentence:
#     spacy_make_ngrams(sentence, matcher, 'bigram')
# )


In [163]:
'bigram_patterns' in matcher

False

In [109]:
matcher.add(f'bigram_patterns', [[{'POS': i} for i in j] for j in bigram_rules])

In [110]:
# # INCASE
# # Spacy Trigrams
# matcher = Matcher(nlp.vocab)

# trigram_rules = [
#     ['VERB', 'ADJ', 'NOUN'],
#     ['NOUN', 'VERB', 'ADV'],
#     ['NOUN', 'ADP', 'NOUN'],
#     # more rules here...
# ]

# bigram_patterns = [[{'POS': i} for i in j] for j in bigram_rules]

# matcher.add('bigram_patterns', bigram_patterns)

# df_jobs['Job Description spacy_2grams'] = df_jobs['Job Description spacy_sentencized'].apply(
#     lambda sentence:
#     spacy_make_ngrams(sentence, matcher, 'bigram')
# )

# df_jobs['Job Description spacy_2grams_in_sent'] = df_jobs['Job Description spacy_sentencized'].replace(
#     regex = {
#         bigram: '_'.join(bigram.split())
#         for bigrams_list in df_jobs['Job Description spacy_2grams'].to_list()
#         for bigram in bigrams_list
#     }
# )

In [243]:
df_jobs['Job Description spacy_2grams_in_sent'][2]

'its purpose is enabling_people and organisations to use our banking services by ensuring a compliant and competitive_kyc ecosystem.'

In [293]:
df_jobs['Job Description nltk_1grams']

0                                             [client]
1    [global, kyc, organisation, part, ing, 's, coo...
2    [purpose, enabling, people, organisations, use...
3    [global, kyc, organisation, first, line, defen...
4    [global, kyc, organisation, working, many, col...
5    [part, managing, supporting, transformation, t...
6    [global, transaction, monitoring, tm, organiza...
7    [sets, standards, provides, content, expertise...
8    [tm, pillar, together, local, tm, experts, pla...
9    [within, tm, pillar, dedicated, teams, in-dept...
Name: Job Description nltk_1grams, dtype: object

In [308]:
from nltk.util import ngrams, bigrams, trigrams
for idx, tokens in df_jobs['Job Description nltk_1grams'].items():
#     print(tokens)
    print(list(ngrams(tokens, 2)))

[]
[('global', 'kyc'), ('kyc', 'organisation'), ('organisation', 'part'), ('part', 'ing'), ('ing', "'s"), ("'s", 'coo'), ('coo', 'domain')]
[('purpose', 'enabling'), ('enabling', 'people'), ('people', 'organisations'), ('organisations', 'use'), ('use', 'banking'), ('banking', 'services'), ('services', 'ensuring'), ('ensuring', 'compliant'), ('compliant', 'competitive'), ('competitive', 'kyc'), ('kyc', 'ecosystem')]
[('global', 'kyc'), ('kyc', 'organisation'), ('organisation', 'first'), ('first', 'line'), ('line', 'defence'), ('defence', 'department'), ('department', 'providing'), ('providing', 'ing'), ('ing', 'business'), ('business', 'functions'), ('functions', 'guidance'), ('guidance', 'standardised'), ('standardised', 'solutions'), ('solutions', 'area'), ('area', 'kyc-related'), ('kyc-related', 'regulations'), ('regulations', 'well'), ('well', 'supporting'), ('supporting', 'realising'), ('realising', 'operational'), ('operational', 'excellence')]
[('global', 'kyc'), ('kyc', 'organis

In [310]:
# NLTK bi and trigrams
from nltk.util import ngrams, bigrams, trigrams
df_jobs['Job Description nltk_1grams'] = df_jobs['Job Description nltk_tokenized']
ngram_dict = {
    'bigram': 2,
    'trigram': 3
}

# NLTK bi and trigrams

ngram_dict = {
    'bigram': 2,
    'trigram': 3
}

for ngram_name, ngram_num in ngram_dict.items():

    df_jobs[f'Job Description nltk_{str(ngram_num)}grams'] = df_jobs['Job Description nltk_1grams'].apply(
        lambda tokens: list(ngrams(tokens, ngram_num))
    )
    
    df_jobs[f'Job Description nltk_{str(ngram_num)}grams_in_sent'] = df_jobs['Job Description spacy_sentencized'].str.lower().replace(
        regex = {
            ' '.join(ngram_tuple): '_'.join(ngram_tuple)
            for ngrams_list in df_jobs[f'Job Description nltk_{str(ngram_num)}grams']
            for ngram_tuple in ngrams_list
        }
    )


In [317]:
x = {}

for ngrams_list in df_jobs[f'Job Description nltk_2grams']:
    for ngram_tuple in ngrams_list:
#         print(ngram_tuple)
        print('_'.join(ngram_tuple))

global_kyc
kyc_organisation
organisation_part
part_ing
ing_'s
's_coo
coo_domain
purpose_enabling
enabling_people
people_organisations
organisations_use
use_banking
banking_services
services_ensuring
ensuring_compliant
compliant_competitive
competitive_kyc
kyc_ecosystem
global_kyc
kyc_organisation
organisation_first
first_line
line_defence
defence_department
department_providing
providing_ing
ing_business
business_functions
functions_guidance
guidance_standardised
standardised_solutions
solutions_area
area_kyc-related
kyc-related_regulations
regulations_well
well_supporting
supporting_realising
realising_operational
operational_excellence
global_kyc
kyc_organisation
organisation_working
working_many
many_colleagues
colleagues_different
different_disciplines
disciplines_geographies
geographies_backgrounds
part_managing
managing_supporting
supporting_transformation
transformation_towards
towards_global
global_organisation
organisation_contribute
contribute_significantly
significantly_real

In [311]:
df_jobs[f'Job Description nltk_2grams'][2]

[('purpose', 'enabling'),
 ('enabling', 'people'),
 ('people', 'organisations'),
 ('organisations', 'use'),
 ('use', 'banking'),
 ('banking', 'services'),
 ('services', 'ensuring'),
 ('ensuring', 'compliant'),
 ('compliant', 'competitive'),
 ('competitive', 'kyc'),
 ('kyc', 'ecosystem')]

In [312]:
df_jobs[f'Job Description nltk_2grams_in_sent'][2]

'its purpose is enabling_people and organisations to use our banking_services by ensuring a compliant and competitive_kyc_ecosystem.'

In [231]:
df_jobs['Job Description spacy_sentencized']

0                                     About Our Client
1    The Global KYC organisation is part of ING's C...
2    Its purpose is Enabling people and organisatio...
3    Our Global KYC organisation is a first line of...
4    In our Global KYC organisation you will be wor...
5    You will be part of managing and supporting th...
6    The Global Transaction Monitoring (TM) organiz...
7    It sets standards and provides content experti...
8    The TM pillar, together with the local TM expe...
9    Within the TM Pillar, we have dedicated teams ...
Name: Job Description spacy_sentencized, dtype: object

In [232]:
# for ngram_name, ngram_num in ngram_dict.items():
df_jobs[f'Job Description nltk_2grams_in_sent'] = df_jobs['Job Description spacy_sentencized'].str.lower().replace(
    regex = {
        ' '.join(ngram_tuple): '_'.join(ngram_tuple)
        for ngrams_list in df_jobs[f'Job Description nltk_2grams']
        for ngram_tuple in ngrams_list
    }
)

In [290]:
df_jobs[f'Job Description nltk_2grams']

KeyError: 'Job Description nltk_2grams'

In [275]:
df_jobs['Job Description gensim_1grams'] = df_jobs['Job Description gensim_tokenized']
# Gensim bi and trigrams
# Gensim Bigrams
bigram = Phraser(Phrases(df_jobs['Job Description gensim_1grams'], connector_words=ENGLISH_CONNECTOR_WORDS, min_count=1, threshold=1))
df_jobs['Job Description gensim_2grams'] = bigram[df_jobs['Job Description gensim_1grams']]

# Gensim Trigrams
trigram = Phraser(Phrases(df_jobs['Job Description gensim_2grams'], connector_words=ENGLISH_CONNECTOR_WORDS, min_count=1, threshold=1))
df_jobs['Job Description gensim_3grams'] = trigram[df_jobs['Job Description gensim_2grams']]


In [276]:
df_jobs[f'Job Description gensim_2grams'][1]

['global_kyc', 'organis', 'ing', 'coo', 'domain']

In [277]:
x = {}
for ngrams_list in df_jobs[f'Job Description gensim_2grams']:
    for ngrams in ngrams_list:
        if '_' in ngrams:
            print(ngrams)
            print(' '.join(ngrams.split('_')))

global_kyc
global kyc
global_kyc
global kyc
line_defenc
line defenc
kyc_relat
kyc relat
global_kyc
global kyc
global_kyc
global kyc
kyc_relat
kyc relat
line_defenc
line defenc
kyc_relat
kyc relat


In [285]:
    df_jobs[f'Job Description gensim_2grams_in_sent'] = df_jobs['Job Description gensim_1grams'].str.lower().replace(
        regex = {
            ' '.join(ngrams.split('_')): ngrams
            for ngrams_list in df_jobs[f'Job Description gensim_2grams']
            for ngrams in ngrams_list
            if '_' in ngrams
        }
    )

In [289]:
df_jobs['Job Description gensim_1grams']

0                                             [client]
1             [global, kyc, organis, ing, coo, domain]
2    [purpos, enabl, peopl, organis, us, bank, serv...
3    [global, kyc, organis, line, defenc, depart, p...
4    [global, kyc, organis, work, colleagu, differ,...
5    [manag, support, transform, global, organis, c...
6    [global, transact, monitor, organ, pillar, glo...
7    [set, standard, provid, content, expertis, imp...
8    [pillar, local, expert, plai, vital, role, ens...
9    [pillar, dedic, team, depth, kyc, knowledg, ex...
Name: Job Description gensim_1grams, dtype: object

In [287]:
df_jobs[f'Job Description gensim_2grams']

0                                             [client]
1              [global_kyc, organis, ing, coo, domain]
2    [purpos, enabl, peopl, organis, us, bank, serv...
3    [global_kyc, organis, line_defenc, depart, pro...
4    [global_kyc, organis, work, colleagu, differ, ...
5    [manag, support, transform, global, organis, c...
6    [global, transact, monitor, organ, pillar, glo...
7    [set, standard, provid, content, expertis, imp...
8    [pillar, local, expert, plai, vital, role, ens...
9    [pillar, dedic, team, depth, kyc, knowledg, ex...
Name: Job Description gensim_2grams, dtype: object

In [288]:
df_jobs[f'Job Description gensim_2grams_in_sent']

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
6   NaN
7   NaN
8   NaN
9   NaN
Name: Job Description gensim_2grams_in_sent, dtype: float64

In [27]:
# df_jobs['Job Description spacy_2grams_in_sent'] = df_jobs['Job Description spacy_2grams_in_sent'].apply(
#     lambda sentence:
#     sentence.split()
# )

In [ ]:
bigram_lists = df_jobs['Job Description spacy_bigram_dicts'].to_list()
bigram_dicts = {}

for bigrams_list in bigram_lists:
    for bigram in bigrams_list:
        bigram_dicts[bigram] = '_'.join(bigram.split())

In [ ]:
bigram_dicts = {
    bigram: '_'.join(bigram.split())
    for bigrams_list in df_jobs['Job Description spacy_bigram_lists'].to_list()
    for bigram in bigrams_list
}

In [ ]:
bigram_dicts

In [ ]:
bigram_dicts

In [ ]:
df_jobs['Job Description spacy_2grams'] = df_jobs['Job Description spacy_sentencized'].replace(
    regex = {
        bigram: '_'.join(bigram.split())
        for bigrams_list in df_jobs['Job Description spacy_bigram_lists'].to_list()
        for bigram in bigrams_list
    }
)


In [ ]:
df_jobs['Job Description spacy_2grams'][2]

In [ ]:
# NLTK bi and trigrams

ngram_dict = {
    'bigram': 2,
    'trigram': 3
}

for ngram_name, ngram_num in ngram_dict.items():

    df_jobs[f'Job Description nltk_{str(ngram_num)}grams'] = df_jobs['Job Description spacy_tokenized'].apply(
        lambda tokens: list(ngrams(tokens, ngram_num))
    )
    
    print(df_jobs[f'Job Description nltk_{str(ngram_num)}grams'])

In [ ]:
df_jobs['Job Description spacy_bigrams'] = df_jobs['Job Description spacy_bigrams'].astype('object')

In [ ]:
df_jobs['Job Description spacy_bigrams']

In [ ]:
x = bigram_dicts.to_list()

In [ ]:
x = {}
for d in bigram_dicts.to_list():
    for k, v in d.items():
        x[k] = v
    x.update(d)

In [ ]:
x = {
    key: value for bigram_dict in bigram_dicts.to_list() for key, value in bigram_dict.items() 
}

In [ ]:
x

In [ ]:
df_jobs['Job Description spacy_bigrams'] = df_jobs['Job Description spacy_sentencized'].replace(
    regex = {
        key: value for bigram_dict in bigram_dicts.to_list() for key, value in bigram_dict.items()
    }
)


In [ ]:
df_jobs['Job Description spacy_bigram'][2]

In [ ]:
df_jobs['Job Description spacy_sentencized'].replace(regex=x)[2]

In [ ]:
print('yes')

In [ ]:
df_jobs['Job Description spacy_bigram_dicts'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda sentence:
    spacy_matches_ngrams(sentence, matcher, 'bigram')
)


In [ ]:
df_jobs['Job Description spacy_bigram_dicts']

In [ ]:
df_jobs['Job Description spacy_bigram'][2]

In [ ]:
df_jobs['Job Description spacy_sentencized'] = df_jobs['Job Description spacy_sentencized'].replace(bigram_dicts)

In [ ]:
df_jobs['Job Description spacy_sentencized']

In [ ]:
bigram_dicts = df_jobs['Job Description spacy_sentencized'].apply(
    lambda sentence:
    spacy_matches_ngrams(sentence, matcher, 'bigram')
)


In [ ]:
bigram_dicts#.to_list()

In [ ]:
d = {}

for dic in bigram_dicts:
    d.update(dic)

In [ ]:
d

In [ ]:
df_jobs['Job Description spacy_sentencized'].replace(d)[2]

In [ ]:
df_jobs['Job Description spacy_bigram_dicts'][2]

In [ ]:
# def spacy_matches_ngrams(sentence, matcher, gram_type):

#     doc = nlp(sentence)
#     matches = {}

#     for match_id, start, end in matcher(doc):
#         if nlp.vocab.strings[match_id].split('_')[0] == gram_type:
#             match = doc[start:end].text
#             matches[match] = '_'.join(match.split())

#     for idx, (key, value) in enumerate(matches.items()):
#         if key in doc.text:
#             print(' '.join(doc.text.split()).replace(key, value))


In [ ]:
df_jobs['Job Description spacy_bigram'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda sentence:
    spacy_matches_ngrams(sentence, matcher, 'bigram')
)


In [ ]:
df_jobs['Job Description spacy_bigram'][2]

In [ ]:
def spacy_matches_ngrams(sentence, matcher, gram_type):
    starts = []
    ends = []
    matches = []
    doc = nlp(sentence)

    for match_id, start, end in matcher(doc):
        if nlp.vocab.strings[match_id].split('_')[0] == gram_type:
            starts.append(start)
            ends.append(end)
    for start, end in zip(starts, ends):
        matches.append(doc[start:end].text)
    new_doc = [doc.text if doc.text.split() not in matches else doc.text.replace(doc.text, '_'.join(match.split()))]
    return new_doc

#         print(f'{doc[:start].text} {matches} {doc[end:].text}')
#     if nlp.vocab.strings[match_id].split('_')[0] == gram_type:
#         matches = '_'.join(doc[start:end].text.split())
#         x.append(matches)
#     print(f'{doc[:start].text} {matches} {doc[end:].text}')
#             return f'{doc[start-start:].text} {matches} {doc[:end].text}'
        
#         new_doc = f'{doc[:start].text} {matches} {doc[end:].text}'
#         x.append(new_doc)
#         print(x)
#             print(f'{doc.text[:start]} {doc.text[:end]}', matches)

            

In [ ]:
df_jobs['Job Description spacy_bigram'][2]

In [ ]:
df_jobs[['Job Description spacy_bigram', 'Job Description spacy_sentencized']]

In [ ]:
# Spacy Trigrams
matcher = Matcher(nlp.vocab)

trigram_rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

trigram_patterns = [[{'POS': i} for i in j] for j in trigram_rules]

matcher.add('trigram_patterns', trigram_patterns)



In [ ]:
df_jobs['Job Description spacy_bigram'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda sentence:
    spacy_matches_ngrams(sentence, matcher, 'bigram')
)


In [ ]:
df_jobs['Job Description spacy_bigram']

In [ ]:
doc = nlp(sentences[4])

In [ ]:
matches = matcher(doc)

print(matches)

In [ ]:
bigram = [
    nlp(sentence)[start:end].text
    for sentence in sentences
    for match_id, start, end in matcher(nlp(sentence))
]

In [ ]:
bigram

In [ ]:
for match_id, start, end in matches:
    
    # Get string representation
    string_id = nlp.vocab.strings[match_id]

    # The matched span
    span = doc[start:end]
    
    print(repr(doc[start:end].text))
    print(match_id, string_id, start, end)
    print()


In [ ]:
# patterns = {
#     "noun_verb": [{"POS": "NOUN"}, {"POS": "VERB"}],
#     "verb_noun": [{"POS": "VERB"}, {"POS": "NOUN"}],
#     "adj_noun": [{"POS": "ADJ"}, {"POS": "NOUN"}],
#     "adj_propn": [{"POS": "ADJ"}, {"POS": "PROPN"}],
# }

# for pattern_name, pattern in bigram_patterns.items():
#     print
# #     matcher.add(pattern_name, [pattern])


In [ ]:
for pattern_name, pattern in trigram_patterns.items():
    matcher.add(pattern_name, [pattern])


In [ ]:
for job_description in job_descriptions:
    doc = nlp(job_description)
    matches = matcher(doc)

In [ ]:
matches

In [ ]:
for job_description in job_descriptions:
    spacy_bigrams = [
        nlp(job_description)[start:end].text
        for _, start, end in matcher(nlp(job_description))

    ]

In [ ]:
for match_id, start, end in matches:
    
    # Get string representation
    string_id = nlp.vocab.strings[match_id]

    # The matched span
    span = doc[start:end]
    
    print(repr(span.text))
    print(match_id, string_id, start, end)
    print()

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


In [ ]:
df_jobs['Job Description spacy_1grams'] = df_jobs['Job Description spacy_tokenized']

In [ ]:
# Spacy bi and trigrams
matcher = Matcher(nlp.vocab)

bigram_rules = [
    ['NOUN', 'VERB'],
    ['VERB', 'NOUN'],
    ['ADJ', 'NOUN'],
    ['ADJ', 'PROPN'],
    # more rules here...
]

trigram_rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

bigram_patterns = [[{'POS': i} for i in j] for j in bigram_rules]
trigram_patterns = [[{'POS': i} for i in j] for j in trigram_rules]

ngram_dict = {
    'bigram': 2,
    'trigram': 3
}

for ngram_name, ngram_num in ngram_dict.items():
    
    matcher.add(f'{ngram_name}_patterns', bigram_patterns)

    df_jobs[f'Job Description spacy_{str(ngram_num)}grams'] = df_jobs['Job Description spacy_sentencized_cleaned'].apply(
        lambda sentence:
        spacy_make_ngrams(sentence, matcher, ngram_name)
    )

    df_jobs[f'Job Description spacy_{str(ngram_num)}grams_in_sent'] = df_jobs['Job Description spacy_sentencized_cleaned'].replace(
        regex = {
            ngram: '_'.join(ngram.split())
            for ngrams_list in df_jobs[f'Job Description spacy_{str(ngram_num)}grams'].to_list()
            for ngram in ngrams_list
        }
    )


In [ ]:
# for sentence in spacy_sentencized:
#     print(model.encode(sentence))



In [ ]:
from transformers.models.electra.modeling_tf_electra import TFElectraMainLayer

In [ ]:
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("pavanchhatpar/electra-base-sentence-splitter")

model = AutoModelForPreTraining.from_pretrained("pavanchhatpar/electra-base-sentence-splitter", from_tf=True)


In [ ]:
nlp = pipeline('sentence-splitter', model)

In [ ]:
from nltk.util import ngrams, bigrams, trigrams

# NLTK Bigrams
df_jobs[f'Job Description nltk_3grams'] = df_jobs['Job Description spacy_tokenized'].apply(
    lambda tokens: list(trigrams(tokens))
)


In [19]:
df_jobs['Job Description gensim_1grams'] = df_jobs['Job Description gensim_tokenized']
# Gensim bi and trigrams
# Gensim Bigrams
bigram = Phraser(Phrases(df_jobs['Job Description gensim_1grams'], connector_words=ENGLISH_CONNECTOR_WORDS, min_count=1, threshold=1))
df_jobs['Job Description gensim_2garms'] = bigram[df_jobs['Job Description gensim_1grams']]

# Gensim Trigrams
trigram = Phraser(Phrases(df_jobs['Job Description gensim_2garms'], connector_words=ENGLISH_CONNECTOR_WORDS, min_count=1, threshold=1))
df_jobs['Job Description gensim_3garms'] = trigram[df_jobs['Job Description gensim_2garms']]


In [20]:
df_jobs['Job Description gensim_3garms']

0                                             [client]
1               [global_kyc_organis, ing, coo, domain]
2    [purpos, enabl, peopl, organis, us, bank, serv...
3    [global_kyc_organis, line_defenc, depart, prov...
4    [global_kyc_organis, work, colleagu, differ, d...
5    [manag, support, transform, global, organis, c...
6    [global, transact, monitor, organ, pillar, glo...
7    [set, standard, provid, content, expertis, imp...
8    [pillar, local, expert, plai, vital, role, ens...
9    [pillar, dedic, team, depth, kyc, knowledg, ex...
Name: Job Description gensim_3garms, dtype: object

In [ ]:
df_jobs[f'Job Description nltk_3grams']

In [ ]:
# INCASE BIGRAMS
# # Spacy Trigrams
# matcher = Matcher(nlp.vocab)

# trigram_rules = [
#     ['VERB', 'ADJ', 'NOUN'],
#     ['NOUN', 'VERB', 'ADV'],
#     ['NOUN', 'ADP', 'NOUN'],
#     # more rules here...
# ]

# trigram_patterns = [[{'POS': i} for i in j] for j in trigram_rules]

# matcher.add('trigram_patterns', trigram_patterns)

# df_jobs['Job Description spacy_3grams'] = df_jobs['Job Description spacy_sentencized'].apply(
#     lambda sentence:
#     spacy_make_ngrams(sentence, matcher, 'trigram')
# )

# df_jobs['Job Description spacy_3grams_in_sent'] = df_jobs['Job Description spacy_sentencized'].replace(
#     regex = {
#         trigram: '_'.join(trigram.split())
#         for trigrams_list in df_jobs['Job Description spacy_3grams'].to_list()
#         for trigram in trigrams_list
#     }
# )


# # NLTK Trigrams
# df_jobs[f'Job Description nltk_3grams'] = df_jobs['Job Description spacy_tokenized'].apply(
#     lambda tokens: list(trigrams(tokens))
# )

In [ ]:
# Gensim bi and trigrams
# Gensim Bigrams
bigram = Phraser(Phrases(df_jobs['Job Description spacy_sentencized'], connector_words=ENGLISH_CONNECTOR_WORDS, min_count=1, threshold=1))
df_jobs['Job Description gensim_2garms'] = bigram[df_jobs['Job Description spacy_tokenized']]

# Gensim Trigrams
trigram = Phraser(Phrases(df_jobs['Job Description gensim_2garms'], connector_words=ENGLISH_CONNECTOR_WORDS, min_count=1, threshold=1))
df_jobs['Job Description gensim_3garms'] = trigram[df_jobs['Job Description gensim_2garms']]


In [ ]:
df_jobs['Job Description gensim_2garms'][2]

In [318]:
import string
import re
import time
import json
import csv
import glob
import pickle
import random
import itertools
import unicodedata
import pandas as pd
import numpy as np
import googletrans
from googletrans import Translator
from collections import defaultdict
random.seed(42)

# Set up Spacy
import spacy
from spacy.symbols import NORM, ORTH, LEMMA, POS
from spacy.matcher import Matcher

nlp = spacy.load('en_core_web_sm')

# Set up NLK
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer, LancasterStemmer
from nltk.tag import pos_tag, pos_tag_sents
from nltk.util import ngrams, bigrams, trigrams

nltk_path = f'{llm_path}/nltk'
nltk.data.path.append(nltk_path)

nltk.download('words', download_dir = nltk_path)
nltk.download('stopwords', download_dir = nltk_path)
nltk.download('punkt', download_dir = nltk_path)
nltk.download('averaged_perceptron_tagger', download_dir = nltk_path)

stop_words = set(stopwords.words('english'))
punctuations = list(string.punctuation)
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Set up Gensim
from gensim.utils import save_as_line_sentence, simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string, preprocess_documents

# Set up Bert
from transformers import AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline, BertTokenizer, BertForPreTraining, BertConfig, BertModel
bert_model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name, strip_accents = True)
bert_model = BertModel.from_pretrained(bert_model_name)


[nltk_data] Downloading package words to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package averaged_perceptron_

In [335]:
df_jobs['Job Description spacy_1grams'] = df_jobs['Job Description spacy_tokenized']

In [ ]:
df_jobs.columns

In [ ]:
df_jobs['Job Description gensim_2grams']

In [330]:
def get_abs_frequency(row, text_col, n_gram_number, embedding_library):

    abs_word_freq = defaultdict(int)
    for word in row[f'Job Description {embedding_library}_{n_gram_number}grams']:
        abs_word_freq[word] += 1

        abs_wtd_df = (
            pd.DataFrame.from_dict(abs_word_freq, orient='index')
            .rename(columns={0: 'abs_word_freq'})
            .sort_values(by=['abs_word_freq'], ascending=False)
            )
        abs_wtd_df.insert(1, 'abs_word_perc', value=abs_wtd_df['abs_word_freq'] / abs_wtd_df['abs_word_freq'].sum())
        abs_wtd_df.insert(2, 'abs_word_perc_cum', abs_wtd_df['abs_word_perc'].cumsum())

        row[f'Job Description {embedding_library}_{n_gram_number}grams_abs_word_freq'] = str(abs_wtd_df['abs_word_freq'].to_dict())
        row[f'Job Description {embedding_library}_{n_gram_number}grams_abs_word_perc'] = str(abs_wtd_df['abs_word_perc'].to_dict())
        row[f'Job Description {embedding_library}_{n_gram_number}grams_abs_word_perc_cum'] = str(abs_wtd_df['abs_word_perc_cum'].to_dict())

    print(row)
    return row


In [320]:
df_jobs['Job Description nltk_1grams'] = df_jobs['Job Description nltk_tokenized']

In [323]:
# NLTK bi and trigrams

ngram_dict = {
    'bigram': 2,
    'trigram': 3
}

for ngram_name, ngram_num in ngram_dict.items():

    df_jobs[f'Job Description nltk_{str(ngram_num)}grams'] = df_jobs['Job Description nltk_1grams'].apply(
        lambda tokens: list(ngrams(tokens, ngram_num))
    )


In [324]:
df_jobs[f'Job Description nltk_2grams']

0                                                   []
1    [(global, kyc), (kyc, organisation), (organisa...
2    [(purpose, enabling), (enabling, people), (peo...
3    [(global, kyc), (kyc, organisation), (organisa...
4    [(global, kyc), (kyc, organisation), (organisa...
5    [(part, managing), (managing, supporting), (su...
6    [(global, transaction), (transaction, monitori...
7    [(sets, standards), (standards, provides), (pr...
8    [(tm, pillar), (pillar, together), (together, ...
9    [(within, tm), (tm, pillar), (pillar, dedicate...
Name: Job Description nltk_2grams, dtype: object

In [325]:
# NLTK Allgrams
df_jobs[f'Job Description nltk_123grams'] = df_jobs['Job Description nltk_1grams'] + df_jobs['Job Description nltk_2grams'] + df_jobs['Job Description nltk_3grams']


In [337]:
# Spacy bi and trigrams
matcher = Matcher(nlp.vocab)

bigram_rules = [
    ['NOUN', 'VERB'],
    ['VERB', 'NOUN'],
    ['ADJ', 'NOUN'],
    ['ADJ', 'PROPN'],
    # more rules here...
]

trigram_rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

patters_dict = {
    'bigram_patterns': [[{'POS': i} for i in j] for j in bigram_rules],
    'trigram_patterns': [[{'POS': i} for i in j] for j in trigram_rules],
}

ngram_dict = {
    'bigram': 2,
    'trigram': 3,
}

for ngram_name, ngram_num in ngram_dict.items():
    
    
    matcher.add(f'{ngram_name}_patterns', patters_dict[f'{ngram_name}_patterns'])

    df_jobs[f'Job Description spacy_{str(ngram_num)}grams'] = df_jobs['Job Description spacy_sentencized'].apply(
        lambda sentence:
        spacy_make_ngrams(sentence, matcher, ngram_name)
    )

    df_jobs[f'Job Description spacy_{str(ngram_num)}grams_in_sent'] = df_jobs['Job Description spacy_sentencized'].str.lower().replace(
        regex = {
            ' '.join(ngram_tuple): '_'.join(ngram_tuple)
            for ngrams_list in df_jobs[f'Job Description spacy_{str(ngram_num)}grams']
            for ngram_tuple in ngrams_list
        }
    )
    
    if f'{ngram_name}_patterns' in matcher:
        matcher.remove(f'{ngram_name}_patterns')
    assert f'{ngram_name}_patterns' not in matcher
df_jobs['Job Description spacy_123grams'] = df_jobs['Job Description spacy_1grams'] + df_jobs['Job Description spacy_2grams'] + df_jobs['Job Description spacy_3grams']


In [338]:
# Gensim bi and trigrams
# Gensim Bigrams
bigram = Phraser(Phrases(df_jobs['Job Description gensim_1grams'], connector_words=ENGLISH_CONNECTOR_WORDS, min_count=1, threshold=1))
df_jobs['Job Description gensim_2grams'] = list(bigram[df_jobs['Job Description gensim_1grams']])

# Gensim Trigrams
trigram = Phraser(Phrases(df_jobs['Job Description gensim_2grams'], connector_words=ENGLISH_CONNECTOR_WORDS, min_count=1, threshold=1))
df_jobs['Job Description gensim_3grams'] = trigram[df_jobs['Job Description gensim_2grams']]
df_jobs[f'Job Description gensim_123grams'] = df_jobs['Job Description gensim_1grams'] + df_jobs['Job Description gensim_2grams'] + df_jobs['Job Description gensim_3grams']


In [339]:
n_grams_list=[1, 2, 3, 123]
embedding_libraries_list = ['spacy', 'nltk', 'gensim']

for embedding_library, n_gram_number in itertools.product(embedding_libraries_list, n_grams_list):
    df_jobs = df_jobs.apply(lambda row: get_abs_frequency(row=row, text_col='Job Description spacy_tokenized', n_gram_number=n_gram_number, embedding_library=embedding_library), axis='columns')


% Sector per Workforce                           0.31385
% per Sector                                    0.001387
% per Social Category                           0.111111
% per Workforce                                 0.000435
Age                                            Mixed Age
                                              ...       
Job Description spacy_2grams                          []
Job Description spacy_2grams_in_sent    about our client
Job Description spacy_3grams                          []
Job Description spacy_3grams_in_sent    about our client
Job Description spacy_123grams                  [client]
Name: 0, Length: 117, dtype: object
% Sector per Workforce                                                            0.31385
% per Sector                                                                     0.001387
% per Social Category                                                            0.111111
% per Workforce                                                    

% Sector per Workforce                                                                        0.31385
% per Sector                                                                                 0.001387
% per Social Category                                                                        0.111111
% per Workforce                                                                              0.000435
Age                                                                                         Mixed Age
                                                                          ...                        
Tracking ID                                                                                       NaN
Type of ownership                                                                                 NaN
Job Description spacy_123grams_abs_word_freq        {'global': 1, 'kyc': 1, 'organisation': 1, 'wo...
Job Description spacy_123grams_abs_word_perc        {'global': 0.09090909090909091

% Sector per Workforce                                                                        0.31385
% per Sector                                                                                 0.001387
% per Social Category                                                                        0.111111
% per Workforce                                                                              0.000435
Age                                                                                         Mixed Age
                                                                          ...                        
Tracking ID                                                                                       NaN
Type of ownership                                                                                 NaN
Job Description spacy_123grams_abs_word_freq        {'purpose': 1, 'enabling': 1, 'people': 1, 'or...
Job Description spacy_123grams_abs_word_perc        {'purpose': 0.0714285714285714

% Sector per Workforce                                                                        0.31385
% per Sector                                                                                 0.001387
% per Social Category                                                                        0.111111
% per Workforce                                                                              0.000435
Age                                                                                         Mixed Age
                                                                          ...                        
Tracking ID                                                                                       NaN
Type of ownership                                                                                 NaN
Job Description spacy_123grams_abs_word_freq        {'teams': 2, 'tm': 1, 'pillar': 1, 'dedicated'...
Job Description spacy_123grams_abs_word_perc        {'teams': 0.14285714285714285,

% Sector per Workforce                                                                        0.31385
% per Sector                                                                                 0.001387
% per Social Category                                                                        0.111111
% per Workforce                                                                              0.000435
Age                                                                                         Mixed Age
                                                                          ...                        
Tracking ID                                                                                       NaN
Type of ownership                                                                                 NaN
Job Description spacy_123grams_abs_word_freq        {'kyc': 2, 'sets': 1, ('including', 'regulator...
Job Description spacy_123grams_abs_word_perc        {'kyc': 0.05714285714285714, '

% Sector per Workforce                                                                        0.31385
% per Sector                                                                                 0.001387
% per Social Category                                                                        0.111111
% per Workforce                                                                              0.000435
Age                                                                                         Mixed Age
                                                                          ...                        
Tracking ID                                                                                       NaN
Type of ownership                                                                                 NaN
Job Description spacy_123grams_abs_word_freq        {'kyc': 2, 'global': 1, ('functions', 'with', ...
Job Description spacy_123grams_abs_word_perc        {'kyc': 0.06896551724137931, '

% Sector per Workforce                                                                        0.31385
% per Sector                                                                                 0.001387
% per Social Category                                                                        0.111111
% per Workforce                                                                              0.000435
Age                                                                                         Mixed Age
                                                                          ...                        
Tracking ID                                                                                       NaN
Type of ownership                                                                                 NaN
Job Description spacy_123grams_abs_word_freq        {'kyc': 2, 'sets': 1, ('including', 'regulator...
Job Description spacy_123grams_abs_word_perc        {'kyc': 0.05714285714285714, '

% Sector per Workforce                                                                        0.31385
% per Sector                                                                                 0.001387
% per Social Category                                                                        0.111111
% per Workforce                                                                              0.000435
Age                                                                                         Mixed Age
                                                                          ...                        
Tracking ID                                                                                       NaN
Type of ownership                                                                                 NaN
Job Description spacy_123grams_abs_word_freq        {'kyc': 2, 'sets': 1, ('including', 'regulator...
Job Description spacy_123grams_abs_word_perc        {'kyc': 0.05714285714285714, '

In [341]:
df_jobs['Job Description gensim_123grams_abs_word_freq']

0                                        {'client': 3}
1    {'ing': 3, 'coo': 3, 'domain': 3, 'organis': 2...
2    {'purpos': 3, 'enabl': 3, 'peopl': 3, 'organis...
3    {'solut': 3, 'busi': 3, 'oper': 3, 'realis': 3...
4    {'work': 3, 'colleagu': 3, 'differ': 3, 'disci...
5    {'manag': 3, 'support': 3, 'transform': 3, 'gl...
6    {'organ': 6, 'global': 4, 'transact': 3, 'moni...
7    {'kyc': 4, 'set': 3, 'standard': 3, 'event': 3...
8    {'pillar': 3, 'effici': 3, 'legisl': 3, 'ctf':...
9    {'team': 9, 'pillar': 3, 'dedic': 3, 'depth': ...
Name: Job Description gensim_123grams_abs_word_freq, dtype: object

In [340]:
df_jobs.columns

Index(['% Sector per Workforce', '% per Sector', '% per Social Category',
       '% per Workforce', 'Age', 'Age_Mixed', 'Age_Num', 'Age_Older',
       'Age_Older (>= 45 years)_% per Sector',
       'Age_Older (>= 45 years)_% per Social Category',
       ...
       'Rating', 'Search Keyword', 'Sector', 'Sector Code',
       'Sector Count (x 1000)', 'Tracking ID', 'Type of ownership',
       'Job Description spacy_123grams_abs_word_freq',
       'Job Description spacy_123grams_abs_word_perc',
       'Job Description spacy_123grams_abs_word_perc_cum'],
      dtype='object', length=126)

In [342]:
df_jobs['Job Description spacy_3grams_abs_word_perc_cum']

0                                                  NaN
1                                                  NaN
2                                                  NaN
3    {('realising', 'operational', 'excellence'): 0...
4                                                  NaN
5                                                  NaN
6                                                  NaN
7    {('provides', 'content', 'expertise'): 0.5, ('...
8                                                  NaN
9                                                  NaN
Name: Job Description spacy_3grams_abs_word_perc_cum, dtype: object

In [546]:
from gensim import corpora, models
from gensim.corpora import Dictionary
from gensim.models import (CoherenceModel, FastText, KeyedVectors, TfidfModel, Word2Vec)

n_grams_list=[1, 2, 3, 123]
embedding_libraries_list = ['spacy', 'nltk']

def get_corpus_and_dictionary(row, ngram_num, embedding_library):

    row[f'Job Description {embedding_library}_{ngram_num}grams_dictionary'] = corpora.Dictionary(
        [
            list(ngram_)
            for ngram_ in row[f'Job Description {embedding_library}_{ngram_num}grams']
            if isinstance(ngram_, tuple) or isinstance(ngram_, str)
            
        ]
    )

#     row[f'Job Description {embedding_library}_{ngram_num}grams_corpus'] = [
#         row[f'Job Description {embedding_library}_{ngram_num}grams_dictionary'].doc2bow(
#             row[f'Job Description {embedding_library}_{ngram_num}grams']
#         )
#     ]

#     row[f'{ngram_num}grams_{embedding_library}_tfidf'] = TfidfModel(
#         row[f'{ngram_num}grams_{embedding_library}_corpus']
#     )

    # row[f'{ngram_num}grams_{embedding_library}_tfidf_matrix'] = [
    #     row[f'{ngram_num}grams_{embedding_library}_tfidf'][doc]
    #     for doc in row[f'{ngram_num}grams_{embedding_library}_corpus']
    # ]

    return row


In [547]:
dataset = ['driving car ',
           'drive car carefully',
           'student and university']

# be sure to split sentence before feed into Dictionary
dataset = [d.split() for d in dataset]
print(dataset)

[['driving', 'car'], ['drive', 'car', 'carefully'], ['student', 'and', 'university']]


In [548]:
df_jobs['Job Description nltk_3grams_dictionary']

0                                                   ()
1                             (0, 1, 2, 3, 4, 5, 6, 7)
2               (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)
3    (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
4                       (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
5    (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
6                             (0, 1, 2, 3, 4, 5, 6, 7)
7    (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
8    (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
9       (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13)
Name: Job Description nltk_3grams_dictionary, dtype: object

In [549]:
for embedding_library, ngram_num in itertools.product(embedding_libraries_list, n_grams_list):
    df_jobs = df_jobs.apply(lambda row: get_corpus_and_dictionary(row=row, ngram_num=ngram_num, embedding_library=embedding_library), axis='columns')



TypeError: decoding to str: need a bytes-like object, list found

In [495]:
df_jobs.columns

Index(['% Sector per Workforce', '% per Sector', '% per Social Category',
       '% per Workforce', 'Age', 'Age_Mixed', 'Age_Num', 'Age_Older',
       'Age_Older (>= 45 years)_% per Sector',
       'Age_Older (>= 45 years)_% per Social Category',
       ...
       'Job Description spacy_123grams_abs_word_perc',
       'Job Description spacy_123grams_abs_word_perc_cum',
       'Job Description spacy_3grams_dictionary',
       'Job Description nltk_3grams_dictionary',
       'Job Description spacy_2grams_dictionary',
       'Job Description spacy_123grams_dictionary',
       'Job Description nltk_2grams_dictionary',
       'Job Description nltk_123grams_dictionary',
       'Job Description spacy_1grams_dictionary',
       'Job Description nltk_1grams_dictionary'],
      dtype='object', length=134)

In [497]:
df_jobs['Job Description nltk_123grams_dictionary']

0                             ()
1                             ()
2                             ()
3    (0, 1, 2, 3, 4, 5, 6, 7, 8)
4                             ()
5                             ()
6                             ()
7             (0, 1, 2, 3, 4, 5)
8                             ()
9                             ()
Name: Job Description nltk_123grams_dictionary, dtype: object

In [481]:
for idx, row in df_jobs.iterrows():
    for ngram_tuple in row['Job Description spacy_3grams']:
        if isinstance(ngram_tuple, tuple):
            ngram_tuple = list(ngram_tuple)
#             print(ngram_tuple)

    
    
    x = [
        list(ngram_)
        if isinstance(ngram_, tuple) or isinstance(ngram_, list)
        else ngram_
        for idx, row in df_jobs.iterrows()
        for ngram_ in row['Job Description spacy_3grams']
        
        
    ]

#     for ngram_list in row['Job Description spacy_3grams']:
#         if isinstance(ngram_list, tuple):
#             print(list(ngram))
#             print(type(list(ngram)))

In [482]:
x

[['realising', 'operational', 'excellence'],
 ['functions', 'with', 'guidance'],
 ['line', 'of', 'defence'],
 ['provides', 'content', 'expertise'],
 ['lines', 'of', 'defence']]

In [408]:
df_jobs['Job Description spacy_3grams'].apply(
    lambda ngram_list: [
        list(ngram_tuple)
        for ngram_tuple in ngram_list
        if isinstance(ngram_tuple, tuple)
    ]
)

0                                                   []
1                                                   []
2                                                   []
3    [[realising, operational, excellence], [functi...
4                                                   []
5                                                   []
6                                                   []
7    [[provides, content, expertise], [lines, of, d...
8                                                   []
9                                                   []
Name: Job Description spacy_3grams, dtype: object